## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition','Department']]
print(y_df.dtypes)


Attrition     object
Department    object
dtype: object


In [5]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Education','Age', 'DistanceFromHome','JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]
X_df = pd.get_dummies(X_df, columns=['OverTime'], dtype=int)

# Show the data types for X_df
print(X_df.dtypes)


Education                  int64
Age                        int64
DistanceFromHome           int64
JobSatisfaction            int64
StockOptionLevel           int64
WorkLifeBalance            int64
YearsAtCompany             int64
YearsSinceLastPromotion    int64
NumCompaniesWorked         int64
OverTime_No                int32
OverTime_Yes               int32
dtype: object


In [6]:
# Split the data into training and testing sets 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)

In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = X_train.astype('int64')
X_test = X_test.astype('int64')

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(1102, 11) (368, 11) (1102, 2) (368, 2)


In [8]:
# Create a StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)


# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
enc_Department = OneHotEncoder(sparse_output=False)


# Fit the encoder to the training data
enc_Department.fit(y_train['Department'].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
encoded_train_Department = enc_Department.transform(y_train['Department'].values.reshape(-1,1))
encoded_test_Department = enc_Department.transform(y_test['Department'].values.reshape(-1,1))

encoded_train_Department[:]

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [10]:
# Create a OneHotEncoder for the Attrition column
from sklearn.preprocessing import OneHotEncoder
enc_Attrition = OneHotEncoder(sparse_output=False)


# Fit the encoder to the training data
enc_Attrition.fit(y_train['Attrition'].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
encoded_train_Attrition = enc_Attrition.transform(y_train['Attrition'].values.reshape(-1,1))
encoded_test_Attrition = enc_Attrition.transform(y_test['Attrition'].values.reshape(-1,1))

encoded_train_Attrition[:]



array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

## Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data
input_features = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_features,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='Shared_1')(input_layer)
shared_layer2 = layers.Dense(32128, activation='relu', name='Shared_2')(shared_layer1)


In [12]:
# Create a branch for Department with a hidden layer and an output layer
# Create the hidden layer
branch_hidden_layer_1 = layers.Dense(32, activation='relu', name = 'hid_dept')(shared_layer2)

# Create the output layer
output_layer = layers.Dense(3, activation='softmax', name = 'out_dept')(branch_hidden_layer_1)


In [13]:
# Create a branch for Attrition with a hidden layer and an output layer
# Create the hidden layer
branch_hidden_layer_2 = layers.Dense(32, activation='relu', name = 'hid_attrition')(shared_layer2)

# Create the output layer
output_layer2 = layers.Dense(2, activation='sigmoid', name = 'out_attrition')(branch_hidden_layer_2)

In [14]:
# Create the model
model = Model(inputs=input_layer, outputs=[output_layer, output_layer2])


# Compile the model
model.compile(optimizer='adam',
              loss={'out_dept': 'categorical_crossentropy', 'out_attrition': 'categorical_crossentropy'},
              metrics={'out_dept':'accuracy', 'out_attrition':'accuracy'})

# Summarize the model
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Shared_1 (Dense)    │ (None, 64)        │        768 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Shared_2 (Dense)    │ (None, 32128)     │  2,088,320 │ Shared_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hid_dept (Dense)    │ (None, 32)        │  1,028,128 │ Shared_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hid_attrition       │ (None, 32)        │  1,028,128 │ Shared_2[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_dept (Dense)    │ (None, 3)         │         99 │ hid_dept[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_attrition       │ (None, 2)         │         66 │ hid_attrition[0]… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,145,509 (15.81 MB)

 Trainable params: 4,145,509 (15.81 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Train the model
model.fit(X_train_scaled, {'out_dept': encoded_train_Department, 'out_attrition': encoded_train_Attrition}, epochs=100, shuffle=True, verbose=2)


Epoch 1/100
35/35 - 2s - 54ms/step - loss: 1.2700 - out_attrition_accuracy: 0.8158 - out_dept_accuracy: 0.6388
Epoch 2/100
35/35 - 1s - 22ms/step - loss: 1.1748 - out_attrition_accuracy: 0.8303 - out_dept_accuracy: 0.6642
Epoch 3/100
35/35 - 1s - 24ms/step - loss: 1.1432 - out_attrition_accuracy: 0.8494 - out_dept_accuracy: 0.6597
Epoch 4/100
35/35 - 1s - 24ms/step - loss: 1.0988 - out_attrition_accuracy: 0.8603 - out_dept_accuracy: 0.6633
Epoch 5/100
35/35 - 1s - 25ms/step - loss: 1.0935 - out_attrition_accuracy: 0.8666 - out_dept_accuracy: 0.6606
Epoch 6/100
35/35 - 1s - 25ms/step - loss: 1.0597 - out_attrition_accuracy: 0.8639 - out_dept_accuracy: 0.6652
Epoch 7/100
35/35 - 1s - 22ms/step - loss: 1.0259 - out_attrition_accuracy: 0.8711 - out_dept_accuracy: 0.6806
Epoch 8/100
35/35 - 1s - 25ms/step - loss: 0.9911 - out_attrition_accuracy: 0.8784 - out_dept_accuracy: 0.6851
Epoch 9/100
35/35 - 1s - 25ms/step - loss: 0.9515 - out_attrition_accuracy: 0.8884 - out_dept_accuracy: 0.6987
E

In [16]:
# Evaluate the model with the testing data
results_test = model.evaluate(X_test_scaled, {'out_dept': encoded_test_Department, 'out_attrition': encoded_test_Attrition}, verbose=2)
print(results_test)


12/12 - 0s - 19ms/step - loss: 5.3829 - out_attrition_accuracy: 0.8342 - out_dept_accuracy: 0.5000
[5.382884979248047, 0.83423912525177, 0.5]


In [17]:
# Print the accuracy for both department and attrition
print(f"Department predictions Accuracy: {results_test[2]}")
print(f"Attrition predictions Accuracy: {results_test[1]}")



Department predictions Accuracy: 0.5
Attrition predictions Accuracy: 0.83423912525177


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy for the Department is 52% and the Attrition accuracy is 80% which means the model can predict the department. There is much room for improvement if the number of epochs could be increased, adding more layers if network amd also tuning up the hyperparameters
2. For the department output layer, I have used softmax function and for the attrition output layer, I have used the sigmoid function. Softmax function, per ChatGPT3.5 explanation, distributes the probabilities across multiple classes ensuring that the sum of all probabilities equals to 1. The sigmoid function on the other hand helps to maps input values within the range 0 to 1, yes/no, true/false, etc.
3. There are multiple ways to improve the model: 
    a. increasing the number of epochs
    b. adding additional layers to the neural network: 
    c. collecting more data or using different data points